In [1]:
import torchio as tio
import os
import nibabel as nib
import numpy as np

In [2]:
source_mask_dir = "../assets/wm_gm_csf_masks"
source_data_dir = "../assets/source_images"

In [3]:
len(os.listdir(source_data_dir)), len(os.listdir(source_mask_dir))

(359, 359)

In [4]:
augmented_source_dir = "../assets/augmented_source_images"

In [5]:
destination_data_dir = "../assets/cc_motion_corrupted"

In [6]:
os.makedirs(augmented_source_dir, exist_ok=True)
os.makedirs(destination_data_dir, exist_ok=True)

In [7]:
motion_transformation = tio.transforms.RandomMotion(
    degrees=5,
    translation=10,
    num_transforms=3
)

In [8]:
output_shape = (256, 288, 288)

crop_or_pad = tio.CropOrPad(output_shape)

In [9]:
source_images = os.listdir(source_data_dir)

In [10]:
def apply_mask(image, mask):
    image_array = image.get_fdata()
    mask_array = mask.get_fdata()

    mask_array = mask_array.astype(bool)

    masked_image_array = image_array * mask_array

    masked_image = nib.Nifti1Image(masked_image_array, image.affine)

    return masked_image

In [11]:
def apply_contrast_inversion(image, mask):
  image_array = image.get_fdata()
  mask_array = mask.get_fdata()
      
  threshold = np.max(mask_array)

  binary_mask = mask_array == threshold

  max_intensity = np.max(image_array[binary_mask])

  inversed_image_array = image_array.copy()
  inversed_image_array[binary_mask] = max_intensity - image_array[binary_mask]

  print(inversed_image_array[:, :, 170][40, 150])

  inversed_image = nib.Nifti1Image(inversed_image_array, image.affine)

  return inversed_image

In [12]:
import time
from tqdm import tqdm

for image in tqdm(source_images, total=len(source_images)):
    print(f"Now processing: {image}")

    if image == ".DS_Store":
        continue
    
    start = time.time()

    three_d_image = nib.load(os.path.join(source_data_dir, image))
    image_mask = nib.load(os.path.join(source_mask_dir, image.split(".")[0] + ".nii"))

    three_d_image = apply_mask(three_d_image, image_mask)

    three_d_image = apply_contrast_inversion(three_d_image, image_mask)

    three_d_image = crop_or_pad(three_d_image)

    transformed_image = motion_transformation(three_d_image)
    nib.save(three_d_image, os.path.join(augmented_source_dir, image))
    nib.save(transformed_image, os.path.join(destination_data_dir, image))

    print(f"Time taken: {time.time() - start}")

  0%|          | 0/359 [00:00<?, ?it/s]

Now processing: CC0001_philips_15_55_M.nii
0.0


  0%|          | 1/359 [00:02<16:12,  2.72s/it]

Time taken: 2.715911626815796
Now processing: CC0002_philips_15_56_M.nii
878.4688711166382


  1%|          | 2/359 [00:04<14:26,  2.43s/it]

...

Time taken: 2.824083089828491
Now processing: CC0053_philips_15_39_F.nii
899.098388671875


 15%|█▍        | 53/359 [02:05<13:11,  2.59s/it]

Time taken: 2.4255263805389404
Now processing: CC0054_philips_15_46_M.nii
838.8493480682373


 15%|█▍        | 53/359 [02:07<12:13,  2.40s/it]


KeyboardInterrupt: 

In [22]:
nib.load(destination_data_dir + "/" +os.listdir(augmented_source_dir)[120]).get_fdata().shape

(256, 288, 288)